<a href="https://colab.research.google.com/github/lmcanavals/pcd/blob/master/07_01_openmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!g++

g++: fatal error: no input files
compilation terminated.


In [4]:
%%file hola.c
#include <stdio.h>
#include <omp.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}

Overwriting hola.c


In [6]:
%%bash
gcc hola.c && ./a.out

Hello, World!


In [21]:
%%file hola2.c
#include <stdio.h>
#include <omp.h>

int main() {
#pragma omp parallel
{
    int id = omp_get_thread_num();
    printf("Hello (%d)", id);
    printf(", World! (%d)\n", id);
}
    return 0;
}

Overwriting hola2.c


In [22]:
%%bash
export OMP_NUM_THREADS=4
gcc -fopenmp hola2.c && ./a.out

Hello (1), World! (1)
Hello (2)Hello (0), World! (2)
, World! (0)
Hello (3), World! (3)


In [23]:
%%file hola3.c
#include <stdio.h>
#include <omp.h>

int main() {
#pragma omp parallel num_threads(6)
{
    int id = omp_get_thread_num();
    printf("Hello (%d)", id);
    printf(", World! (%d)\n", id);
}
    return 0;
}

Writing hola3.c


In [25]:
%%bash
gcc -fopenmp hola3.c && ./a.out

Hello (1), World! (1)
Hello (2), World! (2)
Hello (0), World! (0)
Hello (3), World! (3)
Hello (4), World! (4)
Hello (5), World! (5)


# Trying to calculate Pi

In [71]:
%%file pi1.c
#include <stdio.h>
#include <omp.h>

static long num_steps = 1000000;
double step;

int main() {
    double start = omp_get_wtime();

    double x, pi, sum = 0.0;

    step = 1.0 / (double)num_steps;
    for (int i = 0; i < num_steps; ++i) {
        x = (i + 0.5)*step;
        sum += 4.0 / (1.0 + x*x);
    }

    pi = step * sum;

    double elapsed = omp_get_wtime() - start;

    printf("Pi = %lf\n", pi);
    printf("Time = %lf seconds\n", elapsed);

    return 0;
}

Overwriting pi1.c


In [76]:
!gcc -fopenmp pi1.c && ./a.out

Pi = 3.141593
Time = 0.013846 seconds


## First parallel version

In [81]:
%%file pi2.c
#include <stdio.h>
#include <omp.h>

#define NUM_THREADS 2

static long num_steps = 1000000;
double step;

int main() {
    int nthreads;
    double start = omp_get_wtime();

    double pi, sum[NUM_THREADS];
    step = 1.0 / (double)num_steps;

    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    int id = omp_get_thread_num();
    int nthreads2 = omp_get_num_threads();
    if (id == 0) nthreads = nthreads2;
    sum[id] = 0.0;
    double x;
    for (int i = id; i < num_steps; i += nthreads2) {
        x = (i + 0.5)*step;
        sum[id] += 4.0 / (1.0 + x*x);
    }
}
    pi = 0.0;
    for (int i = 0; i < nthreads; ++i) {
        pi += sum[i];
    }
    pi = pi * step;

    double elapsed = omp_get_wtime() - start;

    printf("Pi = %lf\n", pi);
    printf("Time = %lf seconds\n", elapsed);

    return 0;
}

Overwriting pi2.c


In [84]:
!gcc -fopenmp pi2.c && ./a.out

Pi = 3.141593
Time = 0.018820 seconds


## Another try

In [85]:
%%file pi3.c
#include <stdio.h>
#include <omp.h>

#define PAD 8
#define NUM_THREADS 2

static long num_steps = 1000000;
double step;

int main() {
    int nthreads;
    double start = omp_get_wtime();

    double pi, sum[NUM_THREADS][PAD];
    step = 1.0 / (double)num_steps;

    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    int id = omp_get_thread_num();
    int nthreads2 = omp_get_num_threads();
    if (id == 0) nthreads = nthreads2;
    sum[id][0] = 0.0;
    double x;
    for (int i = id; i < num_steps; i += nthreads2) {
        x = (i + 0.5)*step;
        sum[id][0] += 4.0 / (1.0 + x*x);
    }
}
    pi = 0.0;
    for (int i = 0; i < nthreads; ++i) {
        pi += sum[i][0];
    }
    pi = pi * step;

    double elapsed = omp_get_wtime() - start;

    printf("Pi = %lf\n", pi);
    printf("Time = %lf seconds\n", elapsed);

    return 0;
}

Overwriting pi3.c


In [88]:
!gcc -fopenmp pi3.c && ./a.out

Pi = 3.141593
Time = 0.011812 seconds


## A better try

In [90]:
%%file pi4.c
#include <stdio.h>
#include <omp.h>

#define PAD 8
#define NUM_THREADS 2

static long num_steps = 1000000;
double step;

int main() {
    int nthreads;
    double start = omp_get_wtime();

    double pi;
    step = 1.0 / (double)num_steps;

    omp_set_num_threads(NUM_THREADS);
#pragma omp parallel
{
    int id = omp_get_thread_num();
    int nthreads2 = omp_get_num_threads();
    if (id == 0) nthreads = nthreads2;
    double sum = 0.0;
    double x;
    for (int i = id; i < num_steps; i += nthreads2) {
        x = (i + 0.5)*step;
        sum += 4.0 / (1.0 + x*x);
    }
#pragma omp critical
    pi += sum;
}
    pi = pi * step;

    double elapsed = omp_get_wtime() - start;

    printf("Pi = %lf\n", pi);
    printf("Time = %lf seconds\n", elapsed);

    return 0;
}

Writing pi4.c


In [97]:
!gcc -fopenmp pi4.c && ./a.out

Pi = 3.141593
Time = 0.007248 seconds


## The right way

In [113]:
%%file pi5.c
#include <stdio.h>
#include <omp.h>

static long num_steps = 1000000;
double step;

int main() {
    double start = omp_get_wtime();

    double x, pi, sum = 0.0;

    step = 1.0 / (double)num_steps;
#pragma omp parallel for reduction(+:sum) private(x)
    for (int i = 0; i < num_steps; ++i) {
        x = (i + 0.5)*step;
        sum += 4.0 / (1.0 + x*x);
    }

    pi = step * sum;

    double elapsed = omp_get_wtime() - start;

    printf("Pi = %lf\n", pi);
    printf("Time = %lf seconds\n", elapsed);

    return 0;
}

Overwriting pi5.c


In [114]:
!gcc -fopenmp pi5.c && ./a.out

Pi = 3.141593
Time = 0.007244 seconds
